In [89]:
from sansdata import *

## 0. Compute relative pixel efficiencies from perspex measurement

In [ ]:
sans_file = SansData(f"data/perspex.mpa")

In [ ]:
# Helper function for indexing a rectangle from an array
def rect_slice(center, size):
    x, y = center
    return (slice(y - size // 2, y + size // 2), slice(x - size // 2, x + size // 2))

N_eff_pixel = 540  # just an estimate for now
beamstop_x, beamstop_y = 520, 1024 - 500
beamstop_w, beamstop_h = 76, 76
flood_intensities_full = sans_file.raw_intensity.astype(float)
flood_intensities_full[rect_slice((beamstop_x, beamstop_y),beamstop_w)] = 0
flood_intensities = flood_intensities_full

# Compute pixel efficiency from flood measurement
relative_efficiency = True
if relative_efficiency:
    # Compensate for the fact that the beamstop square is not considered in the average
    area_factor = 1/(1- beamstop_w * beamstop_h/N_eff_pixel ** 2)
    print(area_factor)

    # Idea: compute relative pixel efficiency based on the average pixel counts over the given area
    mu_flood = np.mean(flood_intensities[rect_slice((512, 512), N_eff_pixel)]) * area_factor
    flood_norm = flood_intensities / mu_flood
else:
    # This assumes that the best pixel has efficiency 1? Also not ideal
    max_intensity = np.max(flood_intensities[rect_slice((512, 512), N_eff_pixel)])
    flood_norm = flood_intensities / max_intensity

def plot_image(data, label):
    plt.figure()
    # extent = [550//2, 1024-550//2, 550//2, 1024-550//2]
    extent = [0,1024,0,1024]
    plt.imshow(
        data, cmap="viridis", extent=extent, aspect="auto"
    ) 
    plt.colorbar(label=label)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
plot_image(flood_norm, "Relative efficiency")

## Plot integrals along $x,y$ to see relevance

In [ ]:
# Integrated over x, function of y
proj_y = np.sum(flood_norm, axis = 1)
y_range = np.arange(0,1024,1)
plt.plot(y_range, proj_y)
plt.show()

# Integrated over y, function of x
proj_x = np.sum(flood_norm, axis = 0)
x_range = np.arange(0,1024,1)
plt.plot(x_range, proj_x)
# plt.xlim((400,600))

## Histogram of pixel efficiencies
Verify that the statistics of efficiencies are reasonable: no extraordinarily high values etc.

In [ ]:
# Flatten the 2D array to a 1D array
active_flood = flood_norm[rect_slice((512, 512), N_eff_pixel)]
flat_array = (active_flood).flatten()

# Plot histogram
plt.hist(flat_array, bins=36, edgecolor='black')
plt.xlabel('Pixel efficiency')
plt.ylabel('Frequency')
plt.title('Histogram of efficiency values')
plt.show()

## Synthesize full efficiency map (temporary)
Currently, no flood measurement is available that can characterize the full detector so make up the values in the beamstop area for now by copying other values. NOT FOR USE IN ACTUAL DATA REDUCTIONS.

In [ ]:
flood_norm_synth = np.copy(flood_norm)
offset = 15
flood_norm_synth[rect_slice((beamstop_x, beamstop_y),beamstop_w+offset)] = flood_norm_synth[rect_slice((beamstop_x, beamstop_y+beamstop_w+offset+10),beamstop_w+offset)]
plot_image(flood_norm_synth, "Relative efficiency")

In [83]:
np.savetxt('pixel-efficiency.txt.gz',flood_norm_synth)